In [1]:
import numpy as np

In [2]:
np.random.seed(0)

n = 10_000
f_minus_1 = 3

bias = 4

X = np.random.rand(n, f_minus_1)
X = np.hstack((np.ones((n, 1)), X))
y = X@np.array([[1], [2], [3], [4]]) + np.random.randn(n, 1)


In [3]:
X[:5]

array([[1.        , 0.5488135 , 0.71518937, 0.60276338],
       [1.        , 0.54488318, 0.4236548 , 0.64589411],
       [1.        , 0.43758721, 0.891773  , 0.96366276],
       [1.        , 0.38344152, 0.79172504, 0.52889492],
       [1.        , 0.56804456, 0.92559664, 0.07103606]])

In [38]:
def get_gradient(X_batch, y_batch, w):
    ''' Ritorna il gradiente della loss MSE

    Params
    ------
    X (np.array n x f): sotto-matrice delle features (batch?)
    y (np.array n): targets associati
    w (np.array f): parametri

    Ret
    ---
    gradient (np.array)
    '''
    # vettore colonna
    assert y_batch.ndim == 2 and y_batch.shape[1]==1
    assert w.ndim == 2 and w.shape[1]==1

    return ((w.T@X_batch.T@X_batch)*2 - (y_batch.T@X_batch)*2).T

In [39]:
# generatore
def get_batch(X, y, batch_size):
    ''' Ritorna in sequenza i batch.
    L'ultimo può essere meno numeroso di batch_size

    Params
    ------
    X (np.array n x f): matrice delle features
    y (np.array n): targets
    batch_size: numero di campioni per batch

    Yield
    ------
    (X_batch, y_batch)
    X_batch (np.array batch_size x f): features del batch
    y_batch (np.array batch_size): targets del batch
    '''
    idx = 0
    while idx < X.shape[0]:
        yield(X[idx:idx+batch_size], y[idx:idx+batch_size])
        idx += batch_size

In [50]:
def linear_regression_grad_desc(X, y, w_0, lr, eps, max_epochs, batch_size):
    ''' Esegue un gradient descent

    Params
    ------
    X (np.array n x f): matrice delle features
    y (np.array n): targets
    w_0 (np.array f): parametri iniziali
    lr: learning rate
    eps: convergenza se |w_n-w_(n-1)|< eps
    max_epochs: numero massimo di epoche (si interrompe anche senza convergenza)
    batch_size: numero di campioni per batch

    Ret
    ---
    (convergence, w)
    convergence: bool. Se è stata raggiunta la convergenza
    w: i parametri finali
    '''
    # vettori colonna
    assert y.ndim == 2 and y.shape[1]==1
    assert w_0.ndim == 2 and w_0.shape[1]==1

    convergence = False
    w = np.copy(w_0)

    for epoch in range(max_epochs):
        w_start = np.copy(w)
        for X_batch, y_batch in get_batch(X, y, batch_size):
            gradient = get_gradient(X_batch, y_batch, w)
            w -= gradient * lr
        # finita epoca
        if np.sqrt((w-w_start).T@(w-w_start)) < eps:
            convergence = True
            break

    return convergence, w

In [64]:
conv, w = linear_regression_grad_desc(X, y, w_0=np.zeros((4, 1)), lr=1e-3, eps=1e-15, max_epochs=1000, batch_size=32)

In [65]:
conv

True

In [66]:
w

array([[1.06908631],
       [1.95320149],
       [2.94743193],
       [4.0026106 ]])